In [1]:
#import texthero
import pandas as pd
import numpy as np
import re
import random

### Overview of descriptions, uses(filtered in time/use filter notebook)

In [2]:
df = pd.read_csv('Building_Permits.csv')

/opt/conda/envs/data3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18,22,23,32,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number',
       'Street Number Suffix', 'Street Name', 'Street Suffix', 'Unit',
       'Unit Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Structural Notification',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Voluntary Soft-Story Retrofit', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'TIDF Compliance', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Site Permit',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID', 'SF Fi

In [16]:
df['Description']

0                                        street space permit
1                                    street space & sidewalk
2                                                        NaN
3                                                        NaN
4                                               street space
                                 ...                        
1137687           bath & kitchen remodel, all interior work.
1137688             oti-1 ground floor & basement -  el, me.
1137689    add rooms at ground floor/bdrm and full bath r...
1137690    emergency order #110837-e.  demolish 1-story i...
1137691        replace wall cabinets/replace sink and faucet
Name: Description, Length: 1137692, dtype: object

In [4]:
df['Existing Use'].value_counts().head(5)

1 family dwelling    350320
apartments           205595
office               146329
2 family dwelling    132633
retail sales          43010
Name: Existing Use, dtype: int64

In [15]:
#df['Street Number Suffix'].value_counts()

### Form the addresses in their own Series

In [6]:
df['Street Number Suffix'] = df['Street Number Suffix'].replace(np.NaN,'')
df['Street Suffix'] = df['Street Suffix'].replace(np.NaN,'')

In [7]:

df['Address'] = (df['Street Number'].astype('str') + df['Street Number Suffix'].astype('str')
                    + ' ' + df['Street Name'].astype('str')  + ' ' + df['Street Suffix'].astype('str'))
df['Address'] = df['Address'].apply(lambda x: " ".join(x.split()))

In [8]:
df['Address'][df['Street Number Suffix'].notna()] #checking the appended number suffixes

0             211 Sutter St
1          1170 Guerrero St
2           154 Edinburg St
3              345 Spear St
4              4159 24th St
                 ...       
1137687        999 Green St
1137688     364 Ofarrell St
1137689       1323 Palou Av
1137690        16 Toland St
1137691       2470 Great Hy
Name: Address, Length: 1137692, dtype: object

### Valid Addresses & Standardizing format for Assessor Join
Next, we'll want to verify that the addresses are valid ones. Here is where we'll use the OSM API outlined in JSON and APIs

In [9]:
import requests
def geocode(address):
    params = { 'format'        :'json', 
               'addressdetails': 1, 
               'q'             : address}
    headers = { 'user-agent'   : 'TDI' }   #  Need to supply a user agent other than the default provided 
                                           #  by requests for the API to accept the query.
    return requests.get('http://nominatim.openstreetmap.org/search', params=params, headers=headers)


In [18]:
r_address = random.randint(0,len(df))
r_address, df.loc[r_address, 'Address']

(956335, '2737 Divisadero St')

In [21]:
df.loc[r_address,'Address']+' San Francisco'

'2737 Divisadero St San Francisco'

In [22]:
addr_dict = geocode(df.loc[r_address,'Address']+' street San Francisco').json()[0]['address']

In [27]:
' '.join(addr_dict.values())

'2737 Divisadero Street Pacific Heights San Francisco San Francisco City and County California 94115 United States of America us'

# Uh oh, limited OSM API calls 
Next, you'll want to maybe setup asynchronous requests with requests futures ...
Though the API for nominatim says no threads. This might shut down this approach.

Instead, there are a number of alternative hosts for the 'nominatim' API, with free limits up to ~10k per day

In [ ]:
# non-threaded requests... how to do intelligently?
# need to cache by address, so calling the function on an address seems reasonable

...as well as caching the results maybe by decorating the function call to the API for an address

### Try to 
1. clean up and join, then 
2. see what the results are? 
3. make OSM calls for the remaining values/conflicts and join those?